In [70]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import numpy as np

In [71]:
cid = "aa185ee0f22047cfab9fcae27d390b74"
secret = "dd0f61975fdd4452bb3aa56633d93d71"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [72]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbK4BFAukDzj3?si=2ba85a48be9c4a15" #Top 50 NL 15/02/2022
playlist = sp.playlist_tracks(playlist_link)

tracks = playlist["items"]
track_features = []

for t in tracks:
    track_uri = t["track"]["uri"]
    current_track_features = sp.audio_features(track_uri)[0]
    current_track_features["name"] = t["track"]["name"]
    current_track_features["artists"] = [artist["name"] for artist in t["track"]["artists"]]
    current_track_features["popularity"] = t["track"]["popularity"]
    current_track_features["explicit"] = t["track"]["explicit"]
    current_track_features["album_type"] = t["track"]["album"]["album_type"]
    current_track_features["release_date"] = t["track"]["album"]["release_date"]
    
    artist_genres = []
    for artist in t["track"]["artists"]:
        genres = sp.artist(artist["external_urls"]["spotify"])["genres"]     # extract all genres of an artist
        artist_genres.extend(genres)                                         # combine genres of all artists
    artist_genres = set(artist_genres)                                       # save each genre once
    current_track_features["artist_genres"] = list(artist_genres)
    
    track_features.append(current_track_features)
    
top50NL = pd.DataFrame(track_features)                                                  # make a data frame to export as csv
top50NL = top50NL.drop(columns = ["type", "id", "uri", "track_href", "analysis_url"])   # drop irelevant columns
top50NL.insert(11, 'time_signature', top50NL.pop('time_signature'))                     # make first 12 columns the features

In [73]:
def genre_dummy_creator(sGenre, lSongGenres):
    dummySeries = []
    for songGenres in lSongGenres:
        count = 0
        for genre in songGenres:
            if sGenre in genre:
                count += 1
        if count > 0:
            dummySeries.append(1)
        else:
            dummySeries.append(0)
    return(dummySeries)

for genre in ["dutch", "rap", "pop", "rock", "r&b", "hip hop", "dance"]:
    top50NL[genre] = genre_dummy_creator(genre, top50NL["artist_genres"])  

In [77]:
top50NL.to_csv(r"C:\Users\misko\OneDrive\Desktop\BDS\Block 3\Unsupervised Machine Learning\Week 7\top50NL.csv")
len(top50NL.columns)

27